In [1]:
#Lendo a data de hoje no sistema
from datetime import date
today = date. today()    

#Listando arquivos presentes no diretório
from os import listdir
from os.path import isfile, join
path = 'tables/'

files = [f for f in listdir(path) if isfile(join(path, f))]
if(len(files)==0):
    print('[FRX] There is no files in directory. =(')
else:
    #buscando o arquivo do dia
    try:
        filepath = [s for s in files if 'fii_'+str(today) in s][0]
    except:
        print('[FRX] There is no table today. =(')
        filepath = ''

    if(len(filepath)>0):
        import pandas as pd
        df = pd.read_csv(path+filepath)
        
        #  0 - link                     -
        #  1 - Código do fundo          -
        #  2 - Setor                    -
        #  3 - Preço Atual              -
        #  4 - Liquidez Diária          -
        #  5 - Dividendo                -
        #  6 - Dividend Yield           -
        #  7 - DY (3M) Acumulado        -
        #  8 - DY (6M) Acumulado        -
        #  9 - DY (12M) Acumulado
        # 10 - DY (3M) Média
        # 11 - DY (6M) Média
        # 12 - DY (12M) Média
        # 13 - DY Ano
        # 14 - Variação Preço
        # 15 - Rentab. Período
        # 16 - Rentab. Acumulada
        # 17 - Patrimônio Líq.
        # 18 - VPA
        # 19 - P/VPA
        # 20 - DY Patrimonial
        # 21 - Variação Patrimonial
        # 22 - Rentab. Patr. no Período
        # 23 - Rentab. Patr. Acumulada
        # 24 - Vacância Física
        # 25 - Vacância Financeira
        # 26 - Quantidade Ativos

        #Categorias
        df['Código do fundo'] = df['Código do fundo'].astype('category')
        df['link'] = df['link'].astype('category')
        df['Setor'] = df['Setor'].astype('category')
        
        #int32
        df['Quantidade Ativos'] = df['Quantidade Ativos'].astype('int32')
        
        #float64
        float_mask = [4,19]
        for f in float_mask:
            #df[df.columns[f]] = pd.to_numeric(df[df.columns[f]].str.replacece(',','.',regex=False), errors='coerce')
            df[df.columns[f]] = df[df.columns[f]].apply(lambda x: float(str(x).replace(',','.')))
        
        #moedas
        df['Preço Atual'] = pd.to_numeric(df['Preço Atual'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        df['Dividendo'] = pd.to_numeric(df['Dividendo'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        df['Patrimônio Líq.'] = pd.to_numeric(df['Patrimônio Líq.'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        df['VPA'] = pd.to_numeric(df['VPA'].str.replace('.','', regex=False).str.replace("R$ ","", regex=False).str.replace(',','.',regex=False),errors='coerce')
        
        
        #porcentagens
        percent_mask = [6,7,8,9,10,11,12,13,14,15,16,20,21,22,23,24,25]
        
        for p in percent_mask:
            df[df.columns[p]] = pd.to_numeric(df[df.columns[p]].str.replace("%","",regex=False).str.replace(',','.',regex=False),errors='coerce')


In [2]:
#Distância P/VPA = 1

df['Dif 1'] = abs(df['P/VPA']-1)


#Score
df['Score'] = (
    (df['Dividend Yield ']-df['Dividend Yield '].min())/(df['Dividend Yield '].max()-df['Dividend Yield '].min()))*0.15+(
    (df['DY (12M) Acumulado']-df['DY (12M) Acumulado'].min())/(df['DY (12M) Acumulado'].max()-df['DY (12M) Acumulado'].min()))*0.2+(
    (df['DY (12M) Média']-df['DY (12M) Média'].min())/(df['DY (12M) Média'].max()-df['DY (12M) Média'].min()))*0.3+(
    (df['Rentab. Acumulada']-df['Rentab. Acumulada'].min())/(df['Rentab. Acumulada'].max()-df['Rentab. Acumulada'].min()))*0.1+(
    (df['Liquidez Diária']-df['Liquidez Diária'].min())/(df['Liquidez Diária'].max()-df['Liquidez Diária'].min()))*0.1+(
    1-((df['Dif 1']-df['Dif 1'].min())/(df['Dif 1'].max()-df['Dif 1'].min())))*0.1+(
    (df['Quantidade Ativos']-df['Quantidade Ativos'].min())/(df['Quantidade Ativos'].max()-df['Quantidade Ativos'].min()))*0.05





df.sort_values(by='Score', ascending=False, na_position='last').head(10)

,link,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos,Dif 1,Score
64,/funds/rdes11,RDES11,Lajes Corporativas,64.00,1035.0,10.58,15.50,15.98,16.75,19.14,...,0.78,0.19,-0.04,0.15,1.68,46.74,NaN,4,0.22,0.805327
60,/funds/rbds11,RBDS11,Residencial,70.99,885.0,0.62,0.38,3.38,6.82,17.81,...,0.62,0.00,0.00,0.00,-32.52,NaN,NaN,0,0.38,0.583014
17,/funds/tfof11,TFOF11,Títulos e Val. Mob.,150.45,364.0,1.95,1.32,2.85,8.21,15.10,...,1.17,0.99,0.45,1.44,8.64,NaN,NaN,0,0.17,0.573130
21,/funds/famb11b,FAMB11B,Lajes Corporativas,2990.00,65.0,29.36,0.98,2.91,6.03,14.29,...,0.92,0.89,0.76,1.66,9.67,0.00,NaN,1,0.08,0.562117
20,/funds/brcr11,BRCR11,Híbrido,91.25,43433.0,0.50,0.55,1.40,2.96,14.92,...,0.82,0.45,-0.14,0.31,-1.06,15.30,9.5,11,0.18,0.559648
56,/funds/mfii11,MFII11,Híbrido,121.90,7093.0,1.06,0.87,2.70,5.68,12.66,...,0.96,0.85,26.80,27.88,37.78,NaN,NaN,9,0.04,0.528350
2,/funds/bbfi11b,BBFI11B,Lajes Corporativas,2710.00,162.0,24.57,0.89,2.77,5.63,12.48,...,0.88,0.84,0.01,0.85,6.93,38.59,NaN,2,0.12,0.504761
59,/funds/pord11,PORD11,Títulos e Val. Mob.,105.65,224.0,4.35,3.89,3.89,3.89,10.56,...,1.01,NaN,NaN,NaN,NaN,NaN,NaN,0,0.01,0.480155
104,/funds/xplg11,XPLG11,Logística,105.79,183103.0,0.61,0.57,1.71,3.57,7.52,...,1.08,0.62,-0.62,0.00,3.31,0.00,NaN,6,0.08,0.465912
72,/funds/xpcm11,XPCM11,Lajes Corporativas,57.89,7569.0,0.81,1.38,4.27,6.86,12.18,...,0.69,1.02,0.09,1.11,8.19,0.00,NaN,1,0.31,0.449983


In [3]:
df.describe()

,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,DY (6M) Média,DY (12M) Média,...,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos,Dif 1,Score
count,145.000000,145.00000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,...,145.000000,131.000000,131.000000,131.000000,130.000000,78.000000,11.000000,146.000000,145.000000,145.000000
mean,269.887379,11588.82069,1.808836,0.656164,1.632192,3.057055,6.069521,0.544110,0.509795,0.506027,...,1.058621,0.608626,0.269924,0.880992,7.961231,12.545513,16.503636,3.219178,0.201931,0.326704
std,532.815616,25512.20153,4.201700,1.289284,1.448905,2.004196,3.927682,0.483142,0.333970,0.327476,...,0.309797,0.239772,2.459425,2.524837,38.311277,19.894865,28.764377,7.785480,0.241610,0.111733
min,1.310000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.070000,0.000000,-4.720000,-4.720000,-32.520000,0.000000,0.000000,0.000000,0.000000,0.089389
25%,93.150000,266.00000,0.425000,0.412500,1.160000,2.297500,3.935000,0.385000,0.382500,0.330000,...,0.980000,0.475000,-0.060000,0.435000,2.775000,0.000000,3.150000,0.000000,0.050000,0.265589
50%,109.010000,2105.00000,0.650000,0.550000,1.630000,3.275000,6.580000,0.545000,0.545000,0.550000,...,1.060000,0.610000,0.010000,0.590000,4.770000,4.685000,6.910000,1.000000,0.120000,0.353151
75%,150.450000,9524.00000,0.927500,0.650000,1.930000,3.950000,8.197500,0.640000,0.660000,0.680000,...,1.180000,0.730000,0.140000,0.880000,7.857500,17.872500,13.895000,2.000000,0.290000,0.384592
max,3218.990000,183103.00000,29.360000,15.500000,15.980000,16.750000,19.140000,5.330000,2.790000,1.600000,...,3.070000,1.410000,26.800000,27.880000,432.470000,100.000000,100.000000,64.000000,2.070000,0.805327


In [4]:
df['Score'].hist()

In [5]:
#Lista Desesejos
wish_list =['HGRE11','GGRC11','VILG11','XPML11', 'HFOF11']

carteira_michelle = ['MGFF11','CBOP11','XPIN11','MALL11','RBRF11','FLMA11','KNRI11','VISC11','MXRF11']
carteira_wesley = ['MGFF11','CBOP11','XPIN11','MALL11','RBRF11','FLMA11','HGBS11','HGLG11']

#Lista Negra
black_list = ['XPCM11','PORD11','FAMB11B','RBCB11','TFOF11','BBFI11B','CXCE11B','NVIF11B','CPTS11B','VTLT11','RBVO11','BCIA11','CEOC11','KNIP11','RBBV11','MBRF11']
black_list_setor = ['Hospital','Residencial']

#Intervalo aceitável do P/VPA
pvpa_m = df['P/VPA'].median()
pvpa_des = abs(1-pvpa_m)
pvpa_max = 1 + pvpa_des
pvpa_min = 1 - pvpa_des

#liquidez Diária mínima
ld_min = df['Liquidez Diária'].quantile(0.25)

#DY com tesouro selic
selic_am = 0.54
selic_aa = 6.5

#Vacância mínima (%)
vfisica_max = 10
vfinanc_max = 10

In [6]:
#Filtro
#dff = df[df['P/VPA']<=pvpa_max][df['P/VPA']>=pvpa_min][df['Liquidez Diária']>ld_min][df['Dividend Yield ']>selic_am][df['DY (12M) Acumulado']>selic_aa][~df['Código do fundo'].isin(black_list)][~df['Setor'].isin(black_list_setor)][~(df['Vacância Física']>=10)][~(df['Vacância Financeira']>=10)].sort_values(by='Score', ascending=False, na_position='last')
dff = df[df['Dif 1']<=df['Dif 1'].quantile(0.25)][df['Liquidez Diária']>ld_min][df['Dividend Yield ']>selic_am][df['DY (12M) Acumulado']>selic_aa][~df['Código do fundo'].isin(black_list)][~df['Setor'].isin(black_list_setor)][~(df['Vacância Física']>=10)][~(df['Vacância Financeira']>=10)].sort_values(by='Score', ascending=False, na_position='last')

/home/wesley/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#dff['Preço Compra MAX'] = dff['VPA']*pvpa_max
#dff['Preço Compra MIN'] = dff['VPA']*pvpa_min

dff['Preço Compra MAX'] = dff['VPA']+dff['VPA']*df['Dif 1'].quantile(0.25)
dff['Preço Compra MIN'] = dff['VPA']-dff['VPA']*df['Dif 1'].quantile(0.5)

#dff['Michelle'] = dff['Código do fundo'].isin(carteira_michelle)
#dff['Wesley'] = dff['Código do fundo'].isin(carteira_wesley)
#dff['Desejado'] = dff['Código do fundo'].isin(wish_list)
dff['Carteira'] = dff['Código do fundo'].apply(lambda x: 'Desejado' if x in wish_list else 'Michelle e Wesley' if x in carteira_michelle and x in carteira_wesley else 'Michelle' if x in carteira_michelle else 'Wesley' if x in carteira_wesley else ' - ')

In [8]:
filtro_colunas = ['link',
                  'DY (3M) Acumulado',
                  'DY (6M) Acumulado',
                  'DY (3M) Média',
                  'DY (6M) Média',
                  'DY Ano',
                  'Patrimônio Líq.',
                  'DY Patrimonial',
                  'Variação Patrimonial',
                  'Rentab. Patr. no Período',
                  'Rentab. Patr. Acumulada',
                  'Variação Preço',
                  'Dif 1']

dff = dff.drop(filtro_colunas, axis =1)

# TOP FIIs

In [9]:
dff

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score,Preço Compra MAX,Preço Compra MIN,Carteira
56,MFII11,Híbrido,121.90,7093.0,1.06,0.87,12.66,1.06,3.11,33.30,126.66,0.96,NaN,NaN,9,0.528350,132.9930,111.4608,-
102,HFOF11,Títulos e Val. Mob.,104.68,21929.0,0.60,0.57,9.94,0.83,-0.30,16.45,101.47,1.03,NaN,NaN,0,0.443038,106.5435,89.2936,Desejado
90,OUJP11,Títulos e Val. Mob.,105.90,6205.0,0.65,0.61,10.08,0.84,0.34,7.87,102.24,1.04,NaN,NaN,0,0.431370,107.3520,89.9712,-
92,RBRF11,Títulos e Val. Mob.,103.90,14161.0,0.65,0.62,8.48,0.71,-2.09,8.88,100.61,1.03,NaN,NaN,0,0.395946,105.6405,88.5368,Michelle e Wesley
127,RBED11,Outros,156.10,4589.0,1.00,0.64,8.35,0.70,-7.37,2.90,148.97,1.05,0.0,NaN,2,0.383868,156.4185,131.0936,-
95,BCFF11,Títulos e Val. Mob.,93.15,32177.0,0.57,0.62,7.44,0.62,-1.42,14.32,88.89,1.05,NaN,NaN,0,0.381085,93.3345,78.2232,-
16,FOFT11,Títulos e Val. Mob.,99.75,1395.0,0.70,0.72,8.07,0.67,2.22,8.62,100.99,0.99,NaN,NaN,0,0.378932,106.0395,88.8712,-
116,XPIN11,Outros,114.10,18614.0,0.72,0.63,7.20,0.60,-2.54,0.33,113.72,1.00,3.3,NaN,3,0.361967,119.4060,100.0736,Michelle e Wesley
115,BTCR11,Títulos e Val. Mob.,100.85,2664.0,0.58,0.59,7.46,0.62,-1.59,6.98,97.49,1.03,NaN,NaN,0,0.360443,102.3645,85.7912,-
107,RBRR11,Títulos e Val. Mob.,102.80,17842.0,0.63,0.61,7.15,0.60,-1.97,0.28,97.50,1.05,NaN,NaN,0,0.356033,102.3750,85.8000,-


# Lista de desejos

In [10]:
dfwl = df[df['Código do fundo'].isin(wish_list)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last')
dfwl

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score
102,HFOF11,Títulos e Val. Mob.,104.68,21929.0,0.60,0.57,9.94,0.83,-0.30,16.45,101.47,1.03,NaN,NaN,0,0.443038
94,XPML11,Shoppings,114.00,62773.0,0.57,0.51,6.70,0.56,3.10,11.63,102.97,1.11,2.50,NaN,10,0.380680
89,GGRC11,Logística,139.99,14843.0,0.68,0.47,6.44,0.54,2.84,19.04,117.82,1.19,0.00,0.00,15,0.353151
14,HGRE11,Lajes Corporativas,165.05,20920.0,0.74,0.46,5.85,0.49,6.47,14.72,152.27,1.08,18.73,18.29,20,0.346870
129,VILG11,Logística,115.25,42476.0,0.71,0.59,0.00,0.00,-0.05,27.82,105.70,1.09,0.00,NaN,3,0.202785


# Carteiras

## Michelle

In [11]:
df[df['Código do fundo'].isin(carteira_michelle)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last')

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score
55,MXRF11,Híbrido,11.27,136259.0,0.09,0.80,7.40,0.62,4.13,5.86,9.84,1.15,NaN,NaN,0,0.428188
100,MGFF11,Títulos e Val. Mob.,104.79,19762.0,0.70,0.67,9.32,0.78,3.74,19.44,95.54,1.10,NaN,NaN,0,0.425790
92,RBRF11,Títulos e Val. Mob.,103.90,14161.0,0.65,0.62,8.48,0.71,-2.09,8.88,100.61,1.03,NaN,NaN,0,0.395946
93,VISC11,Shoppings,121.60,36440.0,0.92,0.74,7.05,0.59,9.45,22.05,103.01,1.18,5.80,NaN,10,0.382102
10,CBOP11,Lajes Corporativas,83.55,641.0,0.52,0.61,7.82,0.65,4.05,23.19,78.89,1.06,6.87,6.91,1,0.380195
116,XPIN11,Outros,114.10,18614.0,0.72,0.63,7.20,0.60,-2.54,0.33,113.72,1.00,3.30,NaN,3,0.361967
53,KNRI11,Híbrido,163.00,54313.0,0.74,0.46,5.85,0.49,1.73,11.35,150.81,1.08,2.78,6.54,18,0.361061
96,MALL11,Shoppings,102.03,22761.0,0.56,0.53,7.04,0.59,2.96,3.78,109.61,0.93,0.00,NaN,1,0.357315
33,FLMA11,Híbrido,4.48,54533.0,0.02,0.34,5.27,0.44,7.81,58.70,2.90,1.55,5.49,NaN,1,0.343489


## Wesley

In [12]:
df[df['Código do fundo'].isin(carteira_wesley)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last')

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (12M) Acumulado,DY (12M) Média,Rentab. Período,Rentab. Acumulada,VPA,P/VPA,Vacância Física,Vacância Financeira,Quantidade Ativos,Score
100,MGFF11,Títulos e Val. Mob.,104.79,19762.0,0.70,0.67,9.32,0.78,3.74,19.44,95.54,1.10,NaN,NaN,0,0.425790
92,RBRF11,Títulos e Val. Mob.,103.90,14161.0,0.65,0.62,8.48,0.71,-2.09,8.88,100.61,1.03,NaN,NaN,0,0.395946
10,CBOP11,Lajes Corporativas,83.55,641.0,0.52,0.61,7.82,0.65,4.05,23.19,78.89,1.06,6.87,6.91,1,0.380195
116,XPIN11,Outros,114.10,18614.0,0.72,0.63,7.20,0.60,-2.54,0.33,113.72,1.00,3.30,NaN,3,0.361967
11,HGBS11,Shoppings,257.50,14694.0,1.40,0.55,7.00,0.58,0.44,12.49,216.86,1.19,4.87,NaN,12,0.360025
96,MALL11,Shoppings,102.03,22761.0,0.56,0.53,7.04,0.59,2.96,3.78,109.61,0.93,0.00,NaN,1,0.357315
13,HGLG11,Logística,157.64,32944.0,0.78,0.51,6.06,0.51,1.32,10.60,128.12,1.23,6.80,8.30,14,0.344895
33,FLMA11,Híbrido,4.48,54533.0,0.02,0.34,5.27,0.44,7.81,58.70,2.90,1.55,5.49,NaN,1,0.343489


# Simulador Wesley

In [13]:
#Simulador WESLEY
setores = ['Títulos e Val. Mob.','Outros','Lajes Corporativas','Híbrido','Shoppings','Logística']

#Dados manuais
saldo_disponivel = 3943.94
carteira = [3323.52, 234.20, 891.22, 33.68, 2656.90,2566.15]

#Estratégia de carteira
p_carteira = [0.25, 0.05, 0.20, 0.10, 0.20, 0.20]

dcw = pd.DataFrame({'Setor': setores, '%': p_carteira, 'Investido': carteira})
dcw['Distribuição Ideal'] = round((saldo_disponivel+dcw['Investido'].sum())*dcw['%'],2)
#dcw['Compra Recomendada'] = dcw['Distribuição Ideal'].apply(lambda x: x-dcw['Investido'] if x-dcw['Investido'] > 0 else 0)
dcw['Comprar'] = dcw['Distribuição Ideal']-dcw['Investido'] >= 0

if dcw[dcw['Distribuição Ideal']-dcw['Investido'] < 0].shape[0]>0:
    dcw.loc[(dcw['Distribuição Ideal']-dcw['Investido'] >= 0), 'Disponível para compra'] = round(saldo_disponivel*(dcw['%']/dcw[dcw['Comprar'] == True]['%'].sum()),2)
else:
    dcw.loc[(dcw['Distribuição Ideal']-dcw['Investido'] >= 0), 'Disponível para compra'] = dcw['Distribuição Ideal']-dcw['Investido']
dcw.loc[(dcw['Distribuição Ideal']-dcw['Investido'] < 0), 'Disponível para compra'] = 0.0

dcw.drop('Comprar', axis = 1)

,Setor,%,Investido,Distribuição Ideal,Disponível para compra
0,Títulos e Val. Mob.,0.25,3323.52,3412.40,88.88
1,Outros,0.05,234.20,682.48,448.28
2,Lajes Corporativas,0.20,891.22,2729.92,1838.70
3,Híbrido,0.10,33.68,1364.96,1331.28
4,Shoppings,0.20,2656.90,2729.92,73.02
5,Logística,0.20,2566.15,2729.92,163.77


In [14]:
dcw['Investido'].sum()

9705.67

# Simulador MICHELLE

In [15]:
#Simulador MICHELLE
setores = ['Títulos e Val. Mob.','Outros','Lajes Corporativas','Híbrido','Shoppings','Logística']


#Dados manuais
saldo_disponivel = 6436.76
carteira = [8652.86, 1735.50, 8502.58, 2458.40, 6901.32, 0.0]

#Estratégia de carteira
p_carteira = [0.25, 0.05, 0.20, 0.10, 0.20, 0.20]

dcm= pd.DataFrame({'Setor': setores, '%': p_carteira, 'Investido': carteira})
dcm['Distribuição Ideal'] = round((saldo_disponivel+dcm['Investido'].sum())*dcm['%'],2)
dcm['Comprar'] = dcm['Distribuição Ideal']-dcm['Investido'] >= 0

if dcm[dcm['Distribuição Ideal']-dcm['Investido'] < 0].shape[0]>0:
    dcm.loc[(dcm['Distribuição Ideal']-dcm['Investido'] >= 0), 'Disponível para compra'] = round(saldo_disponivel*(dcm['%']/dcm[dcm['Comprar'] == True]['%'].sum()),2)
else:
    dcm.loc[(dcm['Distribuição Ideal']-dcm['Investido'] >= 0), 'Disponível para compra'] = dcm['Distribuição Ideal']-dcm['Investido']

dcm.loc[(dcm['Distribuição Ideal']-dcm['Investido'] < 0), 'Disponível para compra'] = 0.0

dcm.drop('Comprar', axis = 1)

,Setor,%,Investido,Distribuição Ideal,Disponível para compra
0,Títulos e Val. Mob.,0.25,8652.86,8671.86,2145.59
1,Outros,0.05,1735.50,1734.37,0.00
2,Lajes Corporativas,0.20,8502.58,6937.48,0.00
3,Híbrido,0.10,2458.40,3468.74,858.23
4,Shoppings,0.20,6901.32,6937.48,1716.47
5,Logística,0.20,0.00,6937.48,1716.47


In [16]:
dcm['Investido'].sum()

28250.660000000003

In [17]:
import xlwt
import xlsxwriter

writer = pd.ExcelWriter('tables/top_fii_' + str(today) + '.xls', engine='xlsxwriter')

dff.to_excel(writer,'top_fii','-','%.2f', index=False, encoding='xlwt')
dfwl.to_excel(writer,'lista_desejos','-','%.2f', index=False, encoding='xlwt')
dcm.drop('Comprar', axis = 1).to_excel(writer,'rec_michelle','-','%.2f', index=False, encoding='xlwt')
dcw.drop('Comprar', axis = 1).to_excel(writer,'rec_wesley','-','%.2f', index=False, encoding='xlwt')
df[df['Código do fundo'].isin(carteira_michelle)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last').to_excel(writer,'carteira_michelle','-','%.2f', index=False, encoding='xlwt')
df[df['Código do fundo'].isin(carteira_wesley)].drop(filtro_colunas, axis = 1).sort_values(by='Score', ascending=False, na_position='last').to_excel(writer,'carteira_wesley','-','%.2f', index=False, encoding='xlwt')

writer.save()